In [ ]:
!pip install accelerate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 297.4/297.4 kB 2.1 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.19.3-py3-none-manylinux1_x86_64.whl (166.0 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-many

In [ ]:
import json
from transformers import T5Tokenizer, T5ForConditionalGeneration
import torch
import accelerate

In [ ]:
tokenizer = T5Tokenizer.from_pretrained("google/flan-t5-small")
model = T5ForConditionalGeneration.from_pretrained("google/flan-t5-small", device_map="auto")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565
Special tokens have

In [ ]:
def evaluate(answer: str, predicted: str):
    return answer.lower() in predicted.lower()

In [ ]:
with open('/content/updated_concept_net.dev.csqa.json', 'r') as file:
    data = json.load(file)

# Model Inference with Information

In [ ]:
from tqdm import tqdm

results = []
correct_count = 0  # Initialize correct answer count
incorrect_count = 0  # Initialize incorrect answer count
total_count = 0  # Initialize the total count of records

for record in tqdm(data):
    total_count += 1  # Increment the total record count

    # Form the input text using both the question and the provided knowledge
    input_text = f"Answer the question based on the provided information: \nQuestion: {record['query']}\n\nInformation: {record['knowledges']}"
    input_ids = tokenizer(input_text, return_tensors="pt").input_ids.to("cuda")  # Ensure tensors are on the correct device

    # Generate output and decode
    outputs = model.generate(input_ids)
    predicted_answer = tokenizer.decode(outputs[0], skip_special_tokens=True)

    # Evaluate the prediction
    correct = evaluate(record['answer'], predicted_answer)

    # Count correct predictions
    if correct:
        correct_count += 1
    else:
        incorrect_count += 1

    # Append results with new fields "ok" and "predicted"
    record['ok'] = 1 if correct else 0
    record['predicted'] = predicted_answer  # Store the model's predicted answer
    results.append(record)

    # Calculate and print the current accuracy
    current_accuracy = correct_count / total_count
    print(f"Current Accuracy after processing {total_count} records: {current_accuracy:.2f}")

# At the end of processing, print the number of correct and incorrect answers
print(f"Total Records Processed: {total_count}")
print(f"Number of Correct Answers: {correct_count}")
print(f"Number of Wrong Answers: {incorrect_count}")
accuracy = correct_count / total_count
print(f"Final Accuracy: {accuracy:.2f}")

# Optionally, write the results to a new JSON file
with open('output_with_knowledge_results.json', 'w') as outfile:
    json.dump(results, outfile, indent=4)
print("Processing complete. Results saved to 'output_with_results.json'.")

  0%|          | 0/1221 [00:00<?, ?it/s]/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1178: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(
  0%|          | 1/1221 [00:08<2:48:52,  8.31s/it]

Current Accuracy after processing 1 records: 1.00


  0%|          | 3/1221 [00:08<40:53,  2.01s/it]  

Current Accuracy after processing 2 records: 0.50
Current Accuracy after processing 3 records: 0.67


  1%|          | 7/1221 [00:09<10:21,  1.95it/s]

...

Current Accuracy after processing 1214 records: 0.20
Current Accuracy after processing 1215 records: 0.20
Current Accuracy after processing 1216 records: 0.20


100%|█████████▉| 1217/1221 [02:58<00:00,  7.23it/s]

Current Accuracy after processing 1217 records: 0.20
Current Accuracy after processing 1218 records: 0.20


100%|██████████| 1221/1221 [02:58<00:00,  6.84it/s]

Current Accuracy after processing 1219 records: 0.20
Current Accuracy after processing 1220 records: 0.20
Current Accuracy after processing 1221 records: 0.20
Total Records Processed: 1221
Number of Correct Answers: 249
Number of Wrong Answers: 972
Final Accuracy: 0.20
Processing complete. Results saved to 'output_with_results.json'.
